## Ejercicio 1: Generación de datos sintéticos de reclamaciones de clientes

En este ejercicio utilizaremos la API de OpenAI para generar reclamaciones de clientes en formato JSON.
Luego, convertiremos los resultados en un DataFrame y aplicaremos un análisis de sentimiento sencillo

In [29]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

#print(type(OPENAI_API_KEY),OPENAI_API_KEY)
#print(type(API_VERSION),API_VERSION)
#print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [2]:
import pandas as pd
import json
import os

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

##### Definimos una función para generar reclamaciones sintéticas
Estamos estableciendo valores como el modelo, la temperatura o el max_tokens. Podemos jugar con ellos para ver cómo cambia la generación

In [3]:
from openai import AzureOpenAI
import time

# Configura el cliente de Azure OpenAI
client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

In [4]:
# Check de metodos del objeto
dir(client)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api_key_provider',
 '_api_version',
 '_azure_ad_token',
 '_azure_ad_token_provider',
 '_azure_deployment',
 '_azure_endpoint',
 '_base_url',
 '_build_headers',
 '_build_request',
 '_calculate_retry_timeout',
 '_client',
 '_configure_realtime',
 '_custom_headers',
 '_custom_query',
 '_default_stream_cls',
 '_enforce_trailing_slash',
 '_get_azure_ad_token',
 '_idempotency_header',
 '_idempotency_key',
 '_make_sse_decoder',
 '_make_status_error',
 '_make_status_error_from_response',
 '_maybe_override_

In [5]:
# Ahora podemos transformar en un vector de embedding cualquier texto, por ejemplo:
response = client.embeddings.create(
    model="text-embedding-ada-002", #modelo seleccionado
    input="Hola" #query
)

# Extract the embedding vector
embedding = response.data[0].embedding

# Get its length (dimension)
length = len(embedding)

print(f"Embedding length: {length}")

print("Primeros 5 posiciones del vector de embedding que representa la palabra:")
embedding[:5]

Embedding length: 1536
Primeros 5 posiciones del vector de embedding que representa la palabra:


[-0.01971340738236904,
 -0.00728311762213707,
 -0.019830981269478798,
 -0.012051437050104141,
 -0.027930593118071556]

In [6]:
# Ahora veamos que metodos tiene chat:
dir(client.chat)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_client',
 '_delete',
 '_get',
 '_get_api_list',
 '_patch',
 '_post',
 '_put',
 '_sleep',
 'completions',
 'with_raw_response',
 'with_streaming_response']

In [7]:
model="gpt-4o"
prompt="Hola"

messages = [
    {
        "role": "system",
        "content": "Actúa como un experto informático"
    },
    {
        "role": "user",
        "content": f"{prompt}"  # ← aquí va entre comillas
    }
]

In [8]:
messages.append({
    'role':'user',
    'content':'Resumen de funcionamiento LLM. Se preciso y escueto.'
})

In [9]:
messages

[{'role': 'system', 'content': 'Actúa como un experto informático'},
 {'role': 'user', 'content': 'Hola'},
 {'role': 'user',
  'content': 'Resumen de funcionamiento LLM. Se preciso y escueto.'}]

In [10]:

completion = client.chat.completions.create(
    model=model, #modelo con el que vamos a interactuar
    messages=messages, #prompt es un parametro en este caso
    temperature=1,
    max_tokens=1000,
)

#Cuando se acaba la sesion, se acaba el servicio, a menos que los mensajes que vamos teniendo los vayamos guardando de alguna manera persistente.


In [11]:
# Ver solo el texto de la respuesta
print(completion.choices[0].message.content)

Un **LLM (Modelo de Lenguaje Extendido)** es una IA entrenada con grandes cantidades de texto para comprender y generar lenguaje humano. Utiliza redes neuronales profundas, como Transformadores (ej. GPT), para procesar secuencias de palabras, identificar patrones, y predecir la siguiente palabra en un contexto dado, permitiendo responder preguntas, traducir, resumir y más. Su desempeño mejora con modelos más grandes y adaptaciones específicas.


In [12]:
import json

print(json.dumps(completion.model_dump(), indent=2, ensure_ascii=False))

{
  "id": "chatcmpl-COIjJFi6UyELXVdDudShFHTajkpZh",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Un **LLM (Modelo de Lenguaje Extendido)** es una IA entrenada con grandes cantidades de texto para comprender y generar lenguaje humano. Utiliza redes neuronales profundas, como Transformadores (ej. GPT), para procesar secuencias de palabras, identificar patrones, y predecir la siguiente palabra en un contexto dado, permitiendo responder preguntas, traducir, resumir y más. Su desempeño mejora con modelos más grandes y adaptaciones específicas.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
       

In [13]:
# Expone la razon del porque se detuvo de generar tokens. 'stop' es poque se detuvo antes de llegar al limite de tokens de respuesta, 'length' cuand se detiene cuandoo ya cumplio el max tokens.
completion.choices[0].finish_reason

'stop'

In [14]:
# Numero de tokens consumidos

completion.model_dump().keys()

dict_keys(['id', 'choices', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 'usage', 'prompt_filter_results'])

In [15]:
#Tokens consumidos
completion.usage.total_tokens

130

In [16]:
completion.choices[0].content_filter_results

{'hate': {'filtered': False, 'severity': 'safe'},
 'protected_material_code': {'filtered': False, 'detected': False},
 'protected_material_text': {'filtered': False, 'detected': False},
 'self_harm': {'filtered': False, 'severity': 'safe'},
 'sexual': {'filtered': False, 'severity': 'safe'},
 'violence': {'filtered': False, 'severity': 'safe'}}

In [17]:

completion = client.chat.completions.create(
    model=model, #modelo con el que vamos a interactuar
    messages=messages, #prompt es un parametro en este caso
    temperature=1,
    max_tokens=1000,
    top_p=0.25
)


print(completion.choices[0].message.content)

Un LLM (Large Language Model) es un modelo de inteligencia artificial entrenado con grandes cantidades de texto para procesar y generar lenguaje natural. Utiliza redes neuronales profundas, como transformadores (e.g., GPT), para predecir la probabilidad de palabras en un contexto dado, permitiendo tareas como generación de texto, traducción, resumen y respuesta a preguntas.


## Clase de python importante - ChatSession

In [18]:
class ChatSession:
    def __init__(self, client, model, system_prompt=None):
        self.client = client
        self.model = model
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})

    def send(self, user_input, temperature=0.9, max_tokens=50):
        self.messages.append({"role": "user", "content": user_input})
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        assistant_reply = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_reply})
        return assistant_reply

In [30]:
chat=ChatSession(client, model, system_prompt="Actua como un experto informatico")

In [31]:
chat.model

'gpt-4o'

In [32]:
chat.send("Resumen ejecutivo: Que es un LLM?")

'Un **LLM** (Large Language Model, por sus siglas en inglés) es un modelo de inteligencia artificial enfocado en el procesamiento del lenguaje natural (NLP, Natural Language Processing). Estos modelos están diseñados para comprender, generar, traduc'

In [33]:
chat.messages

[{'role': 'system', 'content': 'Actua como un experto informatico'},
 {'role': 'user', 'content': 'Resumen ejecutivo: Que es un LLM?'},
 {'role': 'assistant',
  'content': 'Un **LLM** (Large Language Model, por sus siglas en inglés) es un modelo de inteligencia artificial enfocado en el procesamiento del lenguaje natural (NLP, Natural Language Processing). Estos modelos están diseñados para comprender, generar, traduc'}]

In [34]:
chat.send("Resumen ejecutivo: Que es una API?")

'Una **API** (Application Programming Interface, o Interfaz de Programación de Aplicaciones) es un conjunto de reglas, protocolos y herramientas que permiten que diferentes sistemas, aplicaciones o servicios se comuniquen entre sí. En términos simples, una API act'

In [35]:
chat.messages

[{'role': 'system', 'content': 'Actua como un experto informatico'},
 {'role': 'user', 'content': 'Resumen ejecutivo: Que es un LLM?'},
 {'role': 'assistant',
  'content': 'Un **LLM** (Large Language Model, por sus siglas en inglés) es un modelo de inteligencia artificial enfocado en el procesamiento del lenguaje natural (NLP, Natural Language Processing). Estos modelos están diseñados para comprender, generar, traduc'},
 {'role': 'user', 'content': 'Resumen ejecutivo: Que es una API?'},
 {'role': 'assistant',
  'content': 'Una **API** (Application Programming Interface, o Interfaz de Programación de Aplicaciones) es un conjunto de reglas, protocolos y herramientas que permiten que diferentes sistemas, aplicaciones o servicios se comuniquen entre sí. En términos simples, una API act'}]

In [36]:
chat.send("Restricciones de historico contexto en una conversacion con LLM")

'Las restricciones relacionadas con el historial de contexto en una conversación con un modelo de lenguaje grande (LLM, como GPT) se refieren a los límites técnicos y operativos que afectan la capacidad del modelo para recordar y manejar de manera efectiva la información de'

### Volvemos al ejercicio

In [19]:
#Le puedo pasar cualquier prompt, es una funcion para interactuar con el LLM
# Forma sintetica de generar reclamaciones

def generar_reclamaciones(prompt: str, num_muestras: int = 5) -> list:
    """Genera una lista de reclamaciones sintéticas utilizando Azure OpenAI."""
    respuestas = []
    for i in range(num_muestras):
        completion = client.chat.completions.create(
            model="gpt-4o",  # Reemplaza "deployment-name" por el nombre del despliegue configurado en Azure (por ejemplo, "gpt-35-instant")
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=1000,
        )
        # Extraemos el contenido de la respuesta
        texto = completion.choices[0].message.content.strip()
        respuestas.append(texto)
        time.sleep(1)  # Pausa para evitar problemas de rate limit
    return respuestas


##### Y ahora hacemos prompt engineering. Es importante estructurarlo lo mejor posible, como hemos visto en la parte teórica

In [20]:
# Definimos el prompt para generar una reclamación en formato JSON.
prompt_reclamacion = (
    "Genera una reclamación sintética de un cliente de un banco que incluya los siguientes campos: \n"
    "- una fecha aleatoria (en formato YYYY-MM-DD) entre el 1 de enero de 2021 y el 31 de diciembre de 2024\n"
    "- para las fechas aleatorias devuelvelas todas diferentes"
    "- producto sobre el que está reclamando (puede ser tarjeta de crédito, préstamo, hipoteca...)\n"
    "- canal de contacto (puede ser teléfono, email o chat, elige uno aleatoriamente)\n"
    "- nivel urgencia (bajo, medio o alto, establecido según la problemática expuesta o aleatoriamente)\n"
    "- descripción (descripción larga y detallada del problema)\n"
    "- Algo de texto extra para que parezca más natural. Puede ser un tono neutro o muy negativo\n"
    "Devuelve la reclamación en formato JSON con las claves: 'fecha', 'producto', 'canal', 'urgencia', 'descripcion'.\n"
    "Escribe únicamente el json de la reclamación, sin ningún preámbulo ni texto extra antes o después."
)

##### Generamos las muestras sintéticas

In [21]:
import json
import re
import pandas as pd

# Generamos las reclamaciones
reclamaciones = generar_reclamaciones(prompt_reclamacion, num_muestras=5)

In [22]:
print("Reclamaciones sintéticas generadas:\n")
for rec in reclamaciones:
    print(rec)
    print("------")

Reclamaciones sintéticas generadas:

```json
{
  "fecha": "2022-07-15",
  "producto": "tarjeta de crédito",
  "canal": "email",
  "urgencia": "alto",
  "descripcion": "Desde hace más de dos semanas he estado intentando resolver un cargo no reconocido en mi tarjeta de crédito, pero no he recibido ninguna solución clara. He enviado varios correos con los detalles del caso y la documentación requerida, pero no obtengo respuesta o solo me mandan mensajes automáticos. Esto está afectando mi confianza en el banco, ya que el monto es considerable y necesito que se solucione con urgencia. Por favor, espero que alguien tome mi caso en serio."
}
```
------
```json
{
  "fecha": "2022-03-15",
  "producto": "tarjeta de crédito",
  "canal": "email",
  "urgencia": "alto",
  "descripcion": "Desde hace más de dos semanas he estado reportando un cargo no reconocido en mi tarjeta de crédito y hasta el momento no he recibido ninguna solución concreta. El monto es considerable y afecta directamente mis fin

In [23]:
reclamaciones[0]

#Ahora vamos a transformar en dataframe

'```json\n{\n  "fecha": "2022-07-15",\n  "producto": "tarjeta de crédito",\n  "canal": "email",\n  "urgencia": "alto",\n  "descripcion": "Desde hace más de dos semanas he estado intentando resolver un cargo no reconocido en mi tarjeta de crédito, pero no he recibido ninguna solución clara. He enviado varios correos con los detalles del caso y la documentación requerida, pero no obtengo respuesta o solo me mandan mensajes automáticos. Esto está afectando mi confianza en el banco, ya que el monto es considerable y necesito que se solucione con urgencia. Por favor, espero que alguien tome mi caso en serio."\n}\n```'

In [24]:
def extraer_json(texto: str) -> str:
    """
    Extrae el bloque JSON del texto eliminando las marcas Markdown si están presentes.
    """
    # Busca un bloque que empiece con ```json y termine con ```
    patron = r"```json\s*(\{.*?\})\s*```"
    coincidencia = re.search(patron, texto, re.DOTALL)
    if coincidencia:
        return coincidencia.group(1)
    else:
        # Si no encuentra las marcas, asume que el texto ya es JSON puro
        return texto.strip()

In [25]:
# Convertimos cada string JSON en un diccionario y creamos un DataFrame
lista_reclamaciones = []
for rec in reclamaciones:
    rec_limpio = extraer_json(rec)
    try:
        diccionario = json.loads(rec_limpio)
        lista_reclamaciones.append(diccionario)
    except json.JSONDecodeError as e:
        print("Error al decodificar JSON:", e)
        print("Contenido problemático:", rec_limpio)

df_reclamaciones = pd.DataFrame(lista_reclamaciones)

print("\nDataFrame de reclamaciones:")
display(df_reclamaciones)


DataFrame de reclamaciones:


,fecha,producto,canal,urgencia,descripcion
0,2022-07-15,tarjeta de crédito,email,alto,Desde hace más de dos semanas he estado intent...
1,2022-03-15,tarjeta de crédito,email,alto,Desde hace más de dos semanas he estado report...
2,2022-06-15,tarjeta de crédito,email,alto,Desde hace más de una semana he estado intenta...
3,2022-03-15,tarjeta de crédito,email,alto,Desde hace más de una semana estoy intentando ...
4,2022-03-15,tarjeta de crédito,email,alto,Desde hace más de dos semanas estoy intentando...


##### - ¿Observas alguna tendencia o patrón en los datos generados?
##### - ¿Cómo podrían ayudar los metadatos (producto, canal, urgencia) a realizar un análisis de sentimiento?

##### Análisis de sentimiento

Una vez hemos generado un dataset, podemos analizar el sentimiento (positivo, neutral, negativo) de las frases para añadirlo como información en el dataframe. Primero, definimos la función y el prompt engineering

In [26]:
def analizar_sentimiento_api(texto: str) -> str:
    """
    Analiza el sentimiento del texto proporcionado utilizando Azure OpenAI.

    Args:
        texto (str): Texto a analizar.

    Returns:
        str: Sentimiento identificado ("Positivo", "Neutral" o "Negativo").
    """
    prompt_sentimiento = (
        "Analiza el siguiente texto y determina su sentimiento. "
        "Responde únicamente con una de las siguientes etiquetas: Positivo, Neutral, Negativo.\n\n"
        f"Texto: {texto}"
    )
    completion = client.chat.completions.create(
        model="gpt-4o",  # Reemplaza con el nombre de tu despliegue en Azure (por ejemplo, "gpt-35-instant")
        messages=[{"role": "user", "content": prompt_sentimiento}],
        temperature=0.1,  # Menor aleatoriedad para respuestas consistentes
        max_tokens=20,    # Queremos una respuesta directa, así que limitamos el máximo de tokens
    )
    sentimiento = completion.choices[0].message.content.strip()
    return sentimiento

In [27]:
df_reclamaciones.head()

,fecha,producto,canal,urgencia,descripcion
0,2022-07-15,tarjeta de crédito,email,alto,Desde hace más de dos semanas he estado intent...
1,2022-03-15,tarjeta de crédito,email,alto,Desde hace más de dos semanas he estado report...
2,2022-06-15,tarjeta de crédito,email,alto,Desde hace más de una semana he estado intenta...
3,2022-03-15,tarjeta de crédito,email,alto,Desde hace más de una semana estoy intentando ...
4,2022-03-15,tarjeta de crédito,email,alto,Desde hace más de dos semanas estoy intentando...


##### Aplicamos el análisis de sentimiento a cada reclamación utilizando la descripción y almacenamos el resultado en una nueva columna 'sentimiento_api

In [28]:
sentimientos = []
for desc in df_reclamaciones['descripcion']:
    sentimiento = analizar_sentimiento_api(desc)
    sentimientos.append(sentimiento)
    time.sleep(1)  # Pausa para evitar rate limits

df_reclamaciones['sentimiento_api'] = sentimientos

print("\nDataFrame con análisis de sentimiento vía API:")
print(df_reclamaciones)


DataFrame con análisis de sentimiento vía API:
        fecha            producto  canal urgencia  \
0  2022-07-15  tarjeta de crédito  email     alto   
1  2022-03-15  tarjeta de crédito  email     alto   
2  2022-06-15  tarjeta de crédito  email     alto   
3  2022-03-15  tarjeta de crédito  email     alto   
4  2022-03-15  tarjeta de crédito  email     alto   

                                         descripcion sentimiento_api  
0  Desde hace más de dos semanas he estado intent...        Negativo  
1  Desde hace más de dos semanas he estado report...        Negativo  
2  Desde hace más de una semana he estado intenta...        Negativo  
3  Desde hace más de una semana estoy intentando ...        Negativo  
4  Desde hace más de dos semanas estoy intentando...        Negativo  


##### - ¿Observas alguna tendencia o patrón en los datos generados y en el análisis de sentimiento obtenido?
##### - ¿Qué diferencias encuentras entre este análisis de sentimiento basado en la API y métodos tradicionales (p.ej., TextBlob)?